## Demo

Алгоритм працює в декілька проходів.

Групи, у яких коефіціент варіації < 20% (задається параметром), вважаються стабільними.

Спочатку намагаємось формувати стабільні групи по 2 мікросервіси. Далі, із тих, що залишились, пробуємо формувати стабільні групи по 3 елементи. І так далі, збільшуємо кількість елементів в групі, допоки не закінчаться елементи, або не досягнемо максимальної кількості елементів (параметр, по дефолту = 4). 


In [12]:
    # Iterate over group sizes from 2 to max_group_size
    for group_size in range(2, min(max_group_size + 1, n + 1)):
        print(f"\nTrying with group size: {group_size}")
        
        if len(available_indices) < group_size:
            print(f"  Not enough services left for group size {group_size}")
            break
        
        # Generate stable groups for current size
        candidate_groups = generate_stable_groups(
            available_indices, 
            microservices, 
            group_size, 
            stability_threshold
        )
        
        if not candidate_groups:
            print(f"  No stable groups found with size {group_size}")
            continue
        
        print(f"  Found {len(candidate_groups)} stable groups with size {group_size}")
        
        # Create set of all already used indices for quick lookup
        used_indices_set = set(idx for group in final_group_indices for idx in group)
        
        # Add stable groups to final groups
        for group, actual_indices, cv in candidate_groups:
            # Skip if any index is already used
            if not is_group_available(actual_indices, used_indices_set):
                continue
                
            # Add this group to final groups
            final_groups.append(group)
            final_group_indices.append(actual_indices)
            final_slot_sums.append(calculate_load_sum(group))
            
            # Update used indices set
            used_indices_set.update(actual_indices)
            
            print(f"  Added group with CV: {cv:.2f}% - {actual_indices}")
        
        # Update available indices by removing used ones
        available_indices = [idx for idx in available_indices if idx not in used_indices_set]
        
        print(f"  Remaining services: {len(available_indices)}")
        
        # If no more services left, we're done
        if not available_indices:
            break

NameError: name 'max_group_size' is not defined

Мікросервіси, що не потрапили в жодну з таких груп, залишаються окремо в свїй групі

In [ ]:
 # Add any remaining microservices as single-element groups
    for idx in available_indices:
        final_groups.append([microservices[idx]])
        final_group_indices.append([idx])
        final_slot_sums.append(calculate_load_sum([microservices[idx]]))
        print(f"  Added single-element group: {idx}")

## Імпорт

In [10]:
import main
import json

## базовий приклад з 6 тайм слотами

In [8]:
print("\n" + "="*60)
print("Input data:")
    
microservices = [
    [1, 2, 3, 4, 3, 2],  # Microservice 0
    [2, 1, 1, 2, 3, 4],  # Microservice 1
    [3, 3, 2, 1, 2, 3],  # Microservice 2
    [4, 3, 2, 1, 1, 2],  # Microservice 3
    [2, 3, 4, 3, 2, 1],  # Microservice 4
    [1, 2, 3, 4, 4, 3],  # Microservice 5
    [3, 2, 1, 2, 3, 4],  # Microservice 6
    [4, 3, 2, 1, 2, 3],  # Microservice 7
]
    
for i, service in enumerate(microservices):
    print(f"Microservice {i}: {service}")

print("="*60 + "\n")

try:
    groups, group_services, slot_sums = main.form_multiple_knapsack_groups(microservices)
    main.print_results(groups, group_services, slot_sums)
except ValueError as e:
    print(f"Error: {e}")
        


Input data:
Microservice 0: [1, 2, 3, 4, 3, 2]
Microservice 1: [2, 1, 1, 2, 3, 4]
Microservice 2: [3, 3, 2, 1, 2, 3]
Microservice 3: [4, 3, 2, 1, 1, 2]
Microservice 4: [2, 3, 4, 3, 2, 1]
Microservice 5: [1, 2, 3, 4, 4, 3]
Microservice 6: [3, 2, 1, 2, 3, 4]
Microservice 7: [4, 3, 2, 1, 2, 3]


Trying with group size: 2
  Found 9 stable groups with size 2
  Added group with CV: 0.00% - [0, 7]
  Added group with CV: 0.00% - [3, 5]
  Added group with CV: 0.00% - [4, 6]
  Remaining services: 2

Trying with group size: 3
  Not enough services left for group size 3
  Added single-element group: 1
  Added single-element group: 2
Grouping results:

Group 1:
  Microservices: [0, 7]
  Individual load by time slots:
    Microservice 0: [1, 2, 3, 4, 3, 2]
    Microservice 7: [4, 3, 2, 1, 2, 3]
  Total load by time slots: [5, 5, 5, 5, 5, 5]
  Average load: 5.00
  Standard deviation: 0.00
  Coefficient of variation: 0.00%

Group 2:
  Microservices: [3, 5]
  Individual load by time slots:
    Microse

## приклад з 24 тайм слотами

(тут генерував дані мікросервісів аі-шкою)

In [9]:
with open('testing/realistic_data.json', 'r') as f:
    real_data = json.load(f)
    
print(f"Завантажено {len(real_data)} мікросервісів з реальними патернами навантаження")

# Запуск алгоритму групування
groups, group_services, slot_sums = main.form_multiple_knapsack_groups(real_data)
print(f"Алгоритм створив {len(groups)} груп")
main.print_results(groups, group_services, slot_sums)

# Аналіз стабільності груп
group_stats = []
for i, (group, services, sums) in enumerate(zip(groups, group_services, slot_sums)):
    mean = sum(sums) / len(sums)
    variance = sum((x - mean) ** 2 for x in sums) / len(sums)
    std_dev = variance ** 0.5
    cv = (std_dev / mean) * 100 if mean > 0 else float('inf')
    
    group_stats.append({
        'group_id': i,
        'size': len(services),
        'mean': mean,
        'std_dev': std_dev,
        'cv': cv,
        'services': services
    })

# Сортування груп за стабільністю (CV)
group_stats.sort(key=lambda x: x['cv'])

# Вивід статистики
print("\nСтатистика груп (відсортована за коефіцієнтом варіації):")
print(f"{'ID групи':<10} {'Розмір':<10} {'Сер. навант.':<15} {'Станд. відх.':<15} {'Коеф. варіації':<15}")
print("-" * 65)

for stat in group_stats[:10]:  # Виводимо тільки 10 найкращих груп
    print(f"{stat['group_id']:<10} {stat['size']:<10} {stat['mean']:.2f}{' '*11} {stat['std_dev']:.2f}{' '*11} {stat['cv']:.2f}%")


Завантажено 50 мікросервісів з реальними патернами навантаження

Trying with group size: 2
  Found 378 stable groups with size 2
  Added group with CV: 0.00% - [13, 47]
  Added group with CV: 0.00% - [16, 44]
  Added group with CV: 0.00% - [18, 48]
  Added group with CV: 0.00% - [30, 42]
  Added group with CV: 0.00% - [31, 40]
  Added group with CV: 0.00% - [36, 49]
  Added group with CV: 0.00% - [9, 43]
  Added group with CV: 0.00% - [8, 46]
  Added group with CV: 0.00% - [7, 41]
  Added group with CV: 0.00% - [38, 45]
  Added group with CV: 5.03% - [3, 4]
  Added group with CV: 5.52% - [0, 2]
  Added group with CV: 6.57% - [1, 6]
  Added group with CV: 14.52% - [15, 25]
  Added group with CV: 14.70% - [17, 27]
  Added group with CV: 15.28% - [23, 26]
  Added group with CV: 15.52% - [10, 34]
  Added group with CV: 15.65% - [22, 28]
  Added group with CV: 15.89% - [20, 32]
  Added group with CV: 15.92% - [11, 33]
  Added group with CV: 16.16% - [21, 29]
  Remaining services: 8

Trying 